<a href="https://colab.research.google.com/github/ShinAsakawa/2019komazawa/blob/master/notebooks/2019komazawa_NST_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center">
    <font  style="font-weight: bolder; " color="darkgreen" size="+2" > 2019年度 駒澤大学心理学特講IIIA 演習教材</font>
</p>

# 絵師になろう :-)  ニューラル画風変換 neural style transfer の実習


<p align="right">
<a href="mailto:asakawa@ieee.org"><font size="+0" style="font-weight:bold" color="RoyalBlue"> 浅川伸一</font></a>
</p>

---

## はじめに

[Gatys et al., (2015)](https://arxiv.org/abs/1508.06576) によって提唱された画風変換を紹介します。<font color="green">ちなみに第一著者の Gatys は発表当時博士課程の学生</font>でした。

彼の考案した画風変換方法は反響を呼び，画像生成，アーティストの関心を集めました。その証拠に [国際会議でコンテンスト](https://nips2017creativity.github.io/) が開催されるほどになりました。
画像生成技術は，[Ian GoodfelIow](https://twitter.com/goodfellow_ian?lang=en)  の [GAN](https://arxiv.org/abs/1406.2661) や [Durk Kingma](https://twitter.com/dpkingma)  の  [VAE](https://arxiv.org/abs/1312.6114)  とともに，技術的な意味ばかりでなく，ニューラルネットワーク内部で何が表現されているのかを考える材料でもあり，心理学者の注目すべき技術であると考えます。(<font color="green">ちなみに Goodfellow も Kingma も論文発表当時博士課程の学生でした</font>)

この文章は，[Harish Narayanan](https://twitter.com/copingbear) の [ブログ](https://harishnarayanan.org/writing/artistic-style-transfer/) と彼の公開した [コード](https://github.com/hnarayanan/artistic-style-transfer)  に基づいています。すぐれたコードを公開してくれた彼に敬意を表します。

画風変換については [Justin Johnson](https://twitter.com/jcjohnss)  が [高速化アルゴリズム](https://arxiv.org/abs/1603.08155)  を発表しだれでもが楽しめる下地ができました。(<font color="green">ちなみに彼もスタンフォード大学の博士課程の学生</font>です。スタンフォード大学の畳込みニューラルネットワークによる画像認識の授業  [cs231](http://cs231n.stanford.edu/) のチューターを務めています。

画像変換のポイントは，2つの損失関数，場合によっては全体変動損失を含めた 3 つの損失関数の和として損失関数を定義することにあります。

1. 内容損失 content loss
2. スタイル損失 style loss
3. 全体変動損失 total variation loss

全体変動損失は用いられない場合もあります。


# 準備作業

必要なライブラリの読み込み

In [0]:
import time
from PIL import Image
import numpy as np

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b

---

内容画像とスタイル画像を読み込みます。
必要に応じて自分の画像に置き換えることができます。

Colab では，Google drive からデータを読み込むために，最初に認証を行う必要があるため，以下の作業が必要になります。
画面に表示される指示に従い，自分の Google アカウントを用いて認証を行ってください。
別タブが開いて認証キーが表示されますので，そのキーを貼り付けてエンターキーを押してください

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# 実習のための画像データの ID を入れて，データを入手
download = drive.CreateFile({'id': '1qETyedlJmViOWPUo3fWdsbtk8hsmJPiZ'})
download.GetContentFile('testimages.zip')

In [0]:
!ls *.zip
!unzip testimages.zip

In [0]:
# 授業中のデモのため画像を小さくして利用しています。時間節約のため
height = 128
width =128

content_image_path = 'withElman.jpg'
content_image = Image.open(content_image_path)
content_image = content_image.resize((width, height))
content_image

In [0]:
style_image_path = 'shinkai.jpg'
style_image = Image.open(style_image_path)
style_image = style_image.resize((width, height))
style_image

In [0]:
content_array = np.asarray(content_image, dtype='float32')
content_array = np.expand_dims(content_array, axis=0)
print(content_array.shape)

style_array = np.asarray(style_image, dtype='float32')
style_array = np.expand_dims(style_array, axis=0)
print(style_array.shape)

In [0]:
content_array[:, :, :, 0] -= 103.939
content_array[:, :, :, 1] -= 116.779
content_array[:, :, :, 2] -= 123.68
content_array = content_array[:, :, :, ::-1]

style_array[:, :, :, 0] -= 103.939
style_array[:, :, :, 1] -= 116.779
style_array[:, :, :, 2] -= 123.68
style_array = style_array[:, :, :, ::-1]

In [0]:
content_image = backend.variable(content_array)
style_image = backend.variable(style_array)
combination_image = backend.placeholder((1, height, width, 3))

In [0]:
input_tensor = backend.concatenate([content_image,
                                    style_image,
                                    combination_image], axis=0)

## 事前学習されたモデルの読み込みと損失関数の定義

The core idea introduced by [Gatys et al. (2015)](https://arxiv.org/abs/1508.06576) is that convolutional neural networks (CNNs) pre-trained for image classification already know how to encode perceptual and semantic information about images. We're going to follow their idea, and use the *feature spaces* provided by one such model to independently work with content and style of images.

The original paper uses the 19 layer VGG network model from [Simonyan and Zisserman (2015)](https://arxiv.org/abs/1409.1556), but we're going to instead follow [Johnson et al. (2016)](https://arxiv.org/abs/1603.08155) and use the 16 layer model (VGG16). There is no noticeable qualitative difference in making this choice, and we gain a tiny bit in speed.

It is trivial for us to get access to this truncated model because Keras comes with a set of pretrained models, including the VGG16 model we're interested in. Note that by setting `include_top=False` in the code below, we don't include any of the fully connected layers.

In [0]:
model = VGG16(input_tensor=input_tensor, weights='imagenet',
              include_top=False)

各層の表示

In [0]:
layers = dict([(layer.name, layer.output) for layer in model.layers])
layers

In [0]:
content_weight = 0.025
style_weight = 5.0
total_variation_weight = 1.0

In [0]:
loss = backend.variable(0.)

### 内容損失

For the content loss, we follow Johnson et al. (2016) and draw the content feature from `block2_conv2`, because the original choice in Gatys et al. (2015) (`block4_conv2`) loses too much structural detail. And at least for faces, I find it more aesthetically pleasing to closely retain the structure of the original content image.

This variation across layers is shown for a couple of examples in the images below (just mentally replace `reluX_Y` with our Keras notation `blockX_convY`).

<!--#![Content feature reconstruction](images/content-feature.png "Content feature reconstruction")
-->
The content loss is the (scaled, squared) Euclidean distance between feature representations of the content and combination images.

In [0]:
def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))

layer_features = layers['block2_conv2']
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss += content_weight * content_loss(content_image_features,
                                      combination_features)

### スタイル損失

画風変換のコアアイデアですが，グラム行列を定義します。通常の損失関数との相違を確認してください

For the style loss, we first define something called a *Gram matrix*. The terms of this matrix are proportional to the covariances of corresponding sets of features, and thus captures information about which features tend to activate together. By only capturing these aggregate statistics across the image, they are blind to the specific arrangement of objects inside the image. This is what allows them to capture information about style independent of content. (This is not trivial at all, and I refer you to [a paper that attempts to explain the idea](https://arxiv.org/abs/1606.01286).)

The Gram matrix can be computed efficiently by reshaping the feature spaces suitably and taking an outer product.


In [0]:
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

The style loss is then the (scaled, squared) Frobenius norm of the difference between the Gram matrices of the style and combination images.

Again, in the following code, I've chosen to go with the style features from layers defined in Johnson et al. (2016) rather than Gatys et al. (2015) because I find the end results more aesthetically pleasing. I encourage you to experiment with these choices to see varying results.

In [0]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

feature_layers = ['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']
for layer_name in feature_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl

### 全体変動損失

Poggio らの標準正則化理論との関連に注目してください

Now we're back on simpler ground.

If you were to solve the optimisation problem with only the two loss terms we've introduced so far (style and content), you'll find that the output is quite noisy. We thus add another term, called the [total variation loss](http://arxiv.org/abs/1412.0035) (a regularisation term) that encourages spatial smoothness.

You can experiment with reducing the `total_variation_weight` and play with the noise-level of the generated image.

In [0]:
def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

loss += total_variation_weight * total_variation_loss(combination_image)

## 損失関数が定義できたので，これを用いて最適化問題を解くことを考えます

The goal of this journey was to setup an optimisation problem that aims to solve for a *combination image* that contains the content of the content image, while having the style of the style image. Now that we have our input images massaged and our loss function calculators in place, all we have left to do is define gradients of the total loss relative to the combination image, and use these gradients to iteratively improve upon our combination image to minimise the loss.

We start by defining the gradients.

In [0]:
grads = backend.gradients(loss, combination_image)

各エポックで損失関数を評価し，その評価した値にしたがって勾配降下法を実行します。

We then introduce an `Evaluator` class that computes loss and gradients in one pass while retrieving them via two separate functions, `loss` and `grads`. This is done because `scipy.optimize` requires separate functions for loss and gradients, but computing them separately would be inefficient.

In [0]:
outputs = [loss]
outputs += grads
f_outputs = backend.function([combination_image], outputs)

def eval_loss_and_grads(x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

L-BFGS を使っていますが，ここでは拘らなくて良いです。
簡単のため 10 回だけ評価しています。

Now we're finally ready to solve our optimisation problem. This combination image begins its life as a random collection of (valid) pixels, and we use the [L-BFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS) algorithm (a quasi-Newton algorithm that's significantly quicker to converge than standard gradient descent) to iteratively improve upon it.

We stop after 10 iterations because the output looks good to me and the loss stops reducing significantly.

In [0]:
x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.

iterations = 10

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

結果の表示までに時間がかかります。気長に待ってください

This took a while on my piddly laptop (that isn't GPU-accelerated), but here is the beautiful output from the last iteration! (Notice that we need to subject our output image to the inverse of the transformation we did to our input images before it makes sense.)

In [0]:
x = x.reshape((height, width, 3))
x = x[:, :, ::-1]
x[:, :, 0] += 103.939
x[:, :, 1] += 116.779
x[:, :, 2] += 123.68
x = np.clip(x, 0, 255).astype('uint8')

Image.fromarray(x)

- That's it.  Enjoy?